### Import Essential packages

In [2]:
import pandas as pd
from re import sub
import nltk
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# stop_words=set(stopwords.words('english'))
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report,accuracy_score,f1_score,roc_auc_score,precision_score,recall_score
import matplotlib.pyplot as plt

In [8]:
df=pd.read_csv("data.csv",encoding='latin1',header=None)
df.columns=['target','ids','date','flag','user','text']
df=df[['target','text']]
print(df)

         target                                               text
0             0  @switchfoot http://twitpic.com/2y1zl - Awww, t...
1             0  is upset that he can't update his Facebook by ...
2             0  @Kenichan I dived many times for the ball. Man...
3             0    my whole body feels itchy and like its on fire 
4             0  @nationwideclass no, it's not behaving at all....
...         ...                                                ...
1599995       4  Just woke up. Having no school is the best fee...
1599996       4  TheWDB.com - Very cool to hear old Walt interv...
1599997       4  Are you ready for your MoJo Makeover? Ask me f...
1599998       4  Happy 38th Birthday to my boo of alll time!!! ...
1599999       4  happy #charitytuesday @theNSPCC @SparksCharity...

[1600000 rows x 2 columns]


### Step 4: Clean the tweets

In [ ]:
def clean_text(text):
    text=text.lower()
    text=sub(r"http\S+|www.\S+","",text)
    text = sub(r"@\w+", "", text)
    text = sub(r"#", "", text)
    text = sub(r"[^\w\s]", "", text)
    # text = " ".join(word for word in text.split() if word not in stop_words)
    return text
df['clean_text'] = df['text'].apply(clean_text)
df.to_csv("cleaned_data.csv")

### Step 5: Split into train/test

In [10]:
df=pd.read_csv("cleaned_data.csv",encoding='latin1')
print(df.head(5))
df.dropna(inplace=True)
x=df['clean_text']
y=df['target']

X_train, X_test, Y_train ,Y_test=train_test_split(x,y,test_size=0.2,random_state=12)
print(Y_train.shape,Y_train.head())
print(X_train.shape,X_train.head())


# Check for null values in X_train and X_test after splitting


                                          clean_text  target
0  awww thats bummer shoulda got david carr third...       0
1  upset cant update facebook texting might cry r...       0
2  dived many times ball managed save 50 rest go ...       0
3                   whole body feels itchy like fire       0
4                           behaving im mad cant see       0
(1274382,) 1195601    4
577876     0
1162719    4
888084     4
1594706    4
Name: target, dtype: int64
(1274382,) 1195601                                 today pretty awesome
577876                                      earache sad face
1162719                                  yay picture changed
888084                       cant w8 cloudy chance meatballs
1594706    cannot hide true feelings intentions soul fami...
Name: clean_text, dtype: object


### Step 6 Text Vectorization


In [4]:
vectorizer=TfidfVectorizer(stop_words='english')
X_train_vector=vectorizer.fit_transform(X_train)
X_test_vector=vectorizer.transform(X_test)

### Step 7 Train a model


In [6]:
#LogisticRegression Model
model =LogisticRegression(max_iter=1000)
model.fit(X_train_vector,Y_train)
Y_pred =model.predict(X_test_vector)
# print(Y_pred)
# print(classification_report(Y_test,Y_pred))
print(f1_score(Y_test,Y_pred,average='weighted'))
print(roc_auc_score(Y_test,Y_pred,average='weighted'))
print(f1_score(Y_test,Y_pred,pos_label=4))
#DecisionTreeModel
# model =DecisionTreeClassifier()
# model.fit(X_train_vector,Y_train)
# Y_pred =model.predict(X_test_vector)
# print(Y_pred)
# print(classification_report(Y_test,Y_pred))
# print(f1_score(Y_test,Y_pred,average='weighted'))
# print(roc_auc_score(Y_test,Y_pred,average='weighted'))
# print(f1_score(Y_test,Y_pred,pos_label=4))


# models={
#     'LogisticRegression':LogisticRegression(max_iter=1000),
#     'DecisionTreeClassifier':DecisionTreeClassifier(),
#     'RandomForestClassifier':RandomForestClassifier(),
#     'GradientBoostingClassifier':GradientBoostingClassifier(),
#     'LinearSCV':LinearSVC(),
# }
# result=pd.DataFrame(columns=['Model','F1 Score','Precision','Recall','ROC-AUC','Accuracy'])
# for name,model in models.items():
#     model.fit(X_train_vector,Y_train)
#     Y_pred =model.predict(X_test_vector)
#     accuracy=accuracy_score(Y_test,Y_pred)
#     f1=f1_score(Y_test,Y_pred,average='weighted')
#     auc=roc_auc_score(Y_test,Y_pred,average='weighted')
#     precision=precision_score(Y_test,Y_pred,average='weighted')
#     recall=recall_score(Y_test,Y_pred,average='weighted')
#     result.loc[name]={'F1 Score':f1,
#                       'Precision':precision,
#                       'Recall':recall,
#                       'ROC-AUC':auc,
#                       'Accuracy':accuracy}




0.7780568604152626
0.7781849777314327
0.7823840737445235


              precision    recall  f1-score   support

           0       0.79      0.75      0.77    160427
           4       0.76      0.80      0.78    159573

    accuracy                           0.78    320000
   macro avg       0.78      0.78      0.77    320000
weighted avg       0.78      0.78      0.77    320000

